In [4]:
from glob import glob
import json
import numpy as np
import os
import tensorflow as tf
from tqdm import tqdm_notebook

print('TensorFlow', tf.__version__)

TensorFlow 2.0.0-rc0


In [2]:
train_image_paths = sorted(
    glob('../../bdd/bdd100k/images/100k/train/*'))
train_label_paths = sorted(
    glob('../../bdd/bdd100k/labels/100k/train/*'))
validation_image_paths = sorted(
    glob('../../bdd/bdd100k/images/100k/val/*'))
validation_label_paths = sorted(
    glob('../../bdd/bdd100k/labels/100k/val/*'))

print('Found training {} images'.format(len(train_image_paths)))
print('Found training {} labels'.format(len(train_label_paths)))
print('Found validation {} images'.format(len(validation_image_paths)))
print('Found validation {} labels'.format(len(validation_label_paths)))

class_map = {value: idx for idx, value in enumerate(['bus',
                                                     'traffic light',
                                                     'traffic sign',
                                                     'person',
                                                     'bike',
                                                     'truck',
                                                     'motor',
                                                     'car',
                                                     'train',
                                                     'rider'])}
for image, label in zip(train_image_paths, train_label_paths):
    assert image.split(
        '/')[-1].split('.')[0] == label.split('/')[-1].split('.')[0]
for image, label in zip(validation_image_paths, validation_label_paths):
    assert image.split(
        '/')[-1].split('.')[0] == label.split('/')[-1].split('.')[0]

Found training 70000 images
Found training 70000 labels
Found validation 10000 images
Found validation 10000 labels


In [5]:
def get_label(label_path, class_map):
    with open(label_path, 'r') as f:
        temp = json.load(f)
    bbox = []
    class_ids = []
    for obj in temp['frames'][0]['objects']:
        if 'box2d' in obj:
            x1 = obj['box2d']['x1']
            y1 = obj['box2d']['y1']
            x2 = obj['box2d']['x2']
            y2 = obj['box2d']['y2']
            bbox.append(np.array([x1, y1, x2, y2]))
            class_ids.append(class_map[obj['category']])
    bbox = np.array(bbox, dtype=np.float32)
    class_ids = np.array(class_ids, dtype=np.float32)[..., None]
    return np.concatenate([bbox, class_ids], axis=-1)

train_labels = []
validation_labels = []

for path in tqdm_notebook(train_label_paths):
    train_labels.append(get_label(path, class_map))
for path in tqdm_notebook(validation_label_paths):
    validation_labels.append(get_label(path, class_map))

In [6]:
n_classes = len(class_map)
input_shape = 512
BATCH_SIZE = 4
EPOCHS = 2
training_steps = len(train_image_paths) // BATCH_SIZE
validation_steps = len(validation_image_paths) // BATCH_SIZE

In [13]:
@tf.function
def get_image(image_path, input_shape=None):
    H = W = input_shape
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, size=[H, W])
    return img


def load_data(input_shape=None):
    def load_data(image_path, label):
        images = get_image(image_path, input_shape=input_shape)
        targets = encode_targets(label, input_shape=input_shape)
        # To-do : transform bbox to account image resizing, add random_flip 
        return images, targets
    return load_data

@tf.function()
def compute_iou(boxes1, boxes2):
    boxes1 = tf.cast(boxes1, dtype=tf.float32)
    boxes2 = tf.cast(boxes2, dtype=tf.float32)

    boxes1_t = change_box_format(boxes1, return_format='x1y1x2y2')
    boxes2_t = change_box_format(boxes2, return_format='x1y1x2y2')

    lu = tf.maximum(boxes1_t[:, None, :2], boxes2_t[:, :2])  # ld ru ??
    rd = tf.minimum(boxes1_t[:, None, 2:], boxes2_t[:, 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[:, :, 0] * intersection[:, :, 1]

    square1 = boxes1[:, 2] * boxes1[:, 3]
    square2 = boxes2[:, 2] * boxes2[:, 3]

    union_square = tf.maximum(square1[:, None] + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)


def change_box_format(boxes, return_format='xywh'):
    boxes = tf.cast(boxes, dtype=tf.float32)
    if return_format == 'xywh':
        # x1 y1 x2 y2
        # 0  1  2  3
        return tf.stack([(boxes[..., 2] + boxes[..., 0]) / 2.0,
                         (boxes[..., 3] + boxes[..., 1]) / 2.0,
                         boxes[..., 2] - boxes[..., 0],
                         boxes[..., 3] - boxes[..., 1]], axis=-1)
    elif return_format == 'x1y1x2y2':
        # x  y  w  h
        # 0  1  2  3
        return tf.stack([boxes[..., 0] - boxes[..., 2] / 2.0,
                         boxes[..., 1] - boxes[..., 3] / 2.0,
                         boxes[..., 0] + boxes[..., 2] / 2.0,
                         boxes[..., 1] + boxes[..., 3] / 2.0], axis=-1)
    return 'You should not be here'


def compute_anchor_dimensions(ratios=[0.5, 1, 2],
                              scales=[1, 1.25, 1.58],
                              areas=[32 * 32, 64 * 64, 128 * 128, 256 * 256, 512 * 512]):
    anchor_shapes = {'P{}'.format(i): [] for i in range(3, 8)}
    for area in areas:
        for ratio in ratios:
            a_h = np.sqrt(area / ratio)
            a_w = area / a_h
            for scale in scales:
                h = np.int32(scale * a_h)
                w = np.int32(scale * a_w)
                anchor_shapes['P{}'.format(
                    int(np.log2(np.sqrt(area) // 4)))].append([w, h])
        anchor_shapes['P{}'.format(int(np.log2(np.sqrt(area) // 4)))] = np.array(
            anchor_shapes['P{}'.format(int(np.log2(np.sqrt(area) // 4)))])
    return anchor_shapes


def get_anchors(input_shape=512, tensor=True):
    anchor_dimensions = compute_anchor_dimensions()
    anchors = []
    for i in range(3, 8):
        feature_name = 'P{}'.format(i)
        stride = 2**i
        feature_size = (input_shape) // stride

        dims = anchor_dimensions[feature_name]
        dims = dims[None, None, ...]
        dims = np.tile(dims, reps=[feature_size, feature_size, 1, 1])

        rx = (np.arange(feature_size) + 0.5) * (stride)
        ry = (np.arange(feature_size) + 0.5) * (stride)
        sx, sy = np.meshgrid(rx, ry)
        cxy = np.stack([sx, sy], axis=-1)
        cxy = cxy[:, :, None, :]
        cxy = np.tile(cxy, reps=[1, 1, 9, 1])
        anchors.append(np.reshape(
            np.concatenate([cxy, dims], axis=-1), [-1, 4]))
    anchors = np.concatenate(anchors, axis=0)
    if tensor:
        anchors = tf.constant(anchors, dtype=tf.float32)
    return anchors


@tf.function
def encode_targets(label, input_shape=None):
    """We use the assignment rule from RPN.
        Faster RCNN box coder follows the coding schema described below:
            ty = (y - ya) / ha
            tx = (x - xa) / wa
            th = log(h / ha)
            tw = log(w / wa)
        where x, y, w, h denote the box's center coordinates, width and height
        respectively. Similarly, xa, ya, wa, ha denote the anchor's center
        coordinates, width and height. tx, ty, tw and th denote the
        anchor-encoded center, width and height respectively.
        See http://arxiv.org/abs/1506.01497 for details.
    """
    anchors = get_anchors(input_shape=input_shape, tensor=True)
    gt_boxes = label[:, :4]
    gt_boxes = change_box_format(gt_boxes, return_format='xywh')
    gt_class_ids = label[:, 4]
    ious = compute_iou(anchors, gt_boxes)

    max_ious = tf.reduce_max(ious, axis=1)
    max_ids = tf.argmax(ious, axis=1, output_type=tf.int32)

    background_mask = max_ious > 0.5
    ignore_mask = tf.logical_and(max_ious > 0.4, max_ious < 0.5)

    selected_gt_boxes = tf.gather(gt_boxes, max_ids)
    selected_gt_class_ids = 1. + tf.gather(gt_class_ids, max_ids)

    """ set achors with iou < 0.5 to 0
        set achors with iou iout > 0.4 && < 0.5 to -1
    """
    selected_gt_class_ids = selected_gt_class_ids * \
        tf.cast(background_mask, dtype=tf.float32)
    classification_targets = selected_gt_class_ids - \
        tf.cast(
            ignore_mask, dtype=tf.float32)
    regression_targets = tf.stack([
        (selected_gt_boxes[:, 0] - anchors[:, 0]) / anchors[:, 2],
        (selected_gt_boxes[:, 1] - anchors[:, 1]) / anchors[:, 3],
        tf.math.log(selected_gt_boxes[:, 2] / anchors[:, 2]),
        tf.math.log(selected_gt_boxes[:, 3] / anchors[:, 3])
    ], axis=-1)
    return (tf.cast(classification_targets, dtype=tf.int32),
            regression_targets,
            background_mask,
            ignore_mask)

In [8]:
def train_data_generator():
    for i in range(len(train_image_paths)):
        yield train_image_paths[i], train_labels[i]


def validation_data_generator():
    for i in range(len(validation_image_paths)):
        yield validation_image_paths[i], validation_labels[i]


def input_fn(training=True, context_id=None):
    def train_input_fn():
        train_dataset = tf.data.Dataset.from_generator(
            train_data_generator, output_types=(tf.string, tf.float32))
        train_dataset = train_dataset.shuffle(1024)
        train_dataset = train_dataset.map(
            load_data(input_shape), num_parallel_calls=tf.data.experimental.AUTOTUNE)
        train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
        train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
        return train_dataset

    def validation_input_fn():
        validation_dataset = tf.data.Dataset.from_generator(
            validation_data_generator, output_types=(tf.string, tf.float32))
        validation_dataset = validation_dataset.map(
            load_data(input_shape), num_parallel_calls=tf.data.experimental.AUTOTUNE)
        validation_dataset = validation_dataset.batch(
            BATCH_SIZE, drop_remainder=True)
        validation_dataset = validation_dataset.prefetch(
            tf.data.experimental.AUTOTUNE)
        return validation_dataset
    if training:
        return train_input_fn
    return validation_input_fn

In [ ]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, n_classes=None):
        self.smooth_l1 = tf.losses.Huber()
        self.num_classes = n_classes

    def focal_loss(self, y_true, y_pred, alpha=0.25, gamma=2):
        y_true = tf.one_hot(y_true, depth=self.num_classes + 1)
        y_true = y_true[:, 1:]
        y_pred = tf.sigmoid(y_pred)

        ce = tf.losses.binary_crossentropy(y_true, y_pred, from_logits=False)
        at = alpha * y_true + (1 - y_true) * (1 - alpha)
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        loss = at * tf.pow(1 - pt, gamma) * tf.expand_dims(ce, axis=1)
        loss = tf.reduce_mean(loss)
        return loss

    def call(self,
             classification_targets,
             classification_predictions,
             regression_targets,
             regression_predictions,
             background_mask,
             ignore_mask):
        num_positive_detections = tf.maximum(tf.reduce_sum(
            tf.cast(background_mask, dtype=tf.float32)), 1.0)
        positive_classification_mask = tf.logical_not(ignore_mask)

        regression_targets_positive = tf.boolean_mask(
            regression_targets, background_mask)
        regression_predictions_positive = tf.boolean_mask(
            regression_predictions, background_mask)

        classification_targets_positive = tf.boolean_mask(
            classification_targets, positive_classification_mask)
        classification_predictions_positive = tf.boolean_mask(
            classification_predictions, positive_classification_mask)

        Lreg = self.smooth_l1(regression_targets_positive,
                              regression_predictions_positive)
        Lcls = self.focal_loss(classification_targets_positive,
                               classification_predictions_positive)
        return Lreg / num_positive_detections, Lcls / num_positive_detections


In [ ]:
import tensorflow as tf


def conv_block(x,
               n_filters,
               size,
               strides=1,
               kernel_init='he_normal',
               bias_init='zeros',
               bn_activated=False):
    x = tf.keras.layers.Conv2D(filters=n_filters,
                               kernel_size=size,
                               padding='same',
                               strides=strides,
                               kernel_initializer=kernel_init,
                               bias_initializer=bias_init)(x)
    if bn_activated:
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
    return x


def Upsampling(tensor, scale=2):
    dims = tensor.shape.as_list()[1:-1]
    return tf.image.resize(tensor, size=[dims[0] * scale, dims[1] * scale])


def build_classification_subnet(n_classes=100, n_anchors=9, p=0.01):
    input_layer = tf.keras.layers.Input(shape=[None, None, 256])
    x = input_layer
    for i in range(4):
        x = conv_block(
            x, 256, 3, kernel_init=tf.keras.initializers.RandomNormal(0.0, 0.01))
        x = tf.keras.layers.ReLU()(x)
    bias_init = -tf.math.log((1 - p) / p).numpy()
    output_layer = tf.keras.layers.Conv2D(filters=n_classes * n_anchors,
                                          kernel_size=3,
                                          padding='same',
                                          kernel_initializer=tf.keras.initializers.RandomNormal(
                                              0.0, 0.01),
                                          bias_initializer=tf.keras.initializers.Constant(
                                              value=bias_init),
                                          activation=None)(x)
    output_layer = tf.keras.layers.Reshape(
        target_shape=[-1, n_classes])(output_layer)
    return tf.keras.Model(inputs=input_layer, outputs=output_layer, name='classification_subnet')


def build_regression_subnet(n_anchors=9):
    input_layer = tf.keras.layers.Input(shape=[None, None, 256])
    x = input_layer
    for i in range(4):
        x = conv_block(
            x, 256, 3, kernel_init=tf.keras.initializers.RandomNormal(0.0, 0.01))
        x = tf.keras.layers.ReLU()(x)
    output_layer = tf.keras.layers.Conv2D(filters=4 * n_anchors,
                                          kernel_size=3,
                                          padding='same',
                                          kernel_initializer=tf.keras.initializers.RandomNormal(
                                              0.0, 0.01),
                                          bias_initializer=tf.keras.initializers.zeros(),
                                          activation=None)(x)
    output_layer = tf.keras.layers.Reshape(target_shape=[-1, 4])(output_layer)
    return tf.keras.Model(inputs=input_layer, outputs=output_layer, name='regression_subnet')


def RetinaNet(input_shape=None, n_classes=None):
    H = W = input_shape
    base_model = tf.keras.applications.ResNet50(
        input_shape=[H, W, 3], weights='imagenet', include_top=False)

    resnet_block_output_names = ['conv3_block4_out',
                                 'conv4_block6_out', 'conv5_block3_out']
    resnet_block_outputs = {'C{}'.format(idx + 3): base_model.get_layer(
        layer).output for idx, layer in enumerate(resnet_block_output_names)}
    resnet_block_outputs = {level: conv_block(
        tensor, 256, 1) for level, tensor in resnet_block_outputs.items()}

    P5 = resnet_block_outputs['C5']
    P6 = conv_block(P5, 256, 3, strides=2)
    P6_relu = tf.keras.layers.ReLU()(P6)
    P7 = conv_block(P6_relu, 256, 3, strides=2)
    M4 = tf.keras.layers.add([tf.keras.layers.Lambda(Upsampling, arguments={'scale': 2})(
        P5), resnet_block_outputs['C4']])
    M3 = tf.keras.layers.add([tf.keras.layers.Lambda(Upsampling, arguments={'scale': 2})(
        M4), resnet_block_outputs['C3']])
    P4 = conv_block(M4, 256, 3)
    P3 = conv_block(M3, 256, 3)
    pyrammid_features = [P7, P6, P5, P4, P3]

    classification_subnet = build_classification_subnet(n_classes=n_classes)
    regression_subnet = build_regression_subnet()

    classification_outputs = [classification_subnet(
        level) for level in pyrammid_features]
    regression_outputs = [regression_subnet(
        level) for level in pyrammid_features]

    classification_head = tf.keras.layers.concatenate(
        classification_outputs, axis=1, name='classification_head')
    regression_head = tf.keras.layers.concatenate(
        regression_outputs, axis=1, name='regression_head')

    return tf.keras.Model(inputs=base_model.input, outputs=[
        classification_head, regression_head],
        name='RetinaNet')

In [ ]:
model = RetinaNet(input_shape=input_shape, n_classes=n_classes)
model.build([None, input_shape, input_shape, 3])
loss_fn = Loss(n_classes=n_classes)
optimizer = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9, decay=1e-4)

model_dir = 'model_files/'
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint_manager = tf.train.CheckpointManager(checkpoint,
                                                directory=model_dir,
                                                max_to_keep=None)

In [9]:
@tf.function
def training_step(batch):
    image, (classification_targets,
            regression_targets,
            background_mask,
            ignore_mask) = batch
    with tf.GradientTape() as tape:
        regression_predictions, classification_predictions = model(
            image, training=True)
        Lreg, Lcls = loss_fn(classification_targets,
                             classification_predictions,
                             regression_targets,
                             regression_predictions,
                             background_mask,
                             ignore_mask)
        total_loss = Lreg + Lcls
    gradients = tape.gradient(total_loss, model.trainable_variables)
    optimizer.apply(zip(gradients, model.trainable_variables))
    loss_dict = {
        'box_loss': Lreg,
        'cls_loss': Lcls,
        'total_loss': total_loss
    }
    return loss_dict


@tf.function
def validation_step(batch):
    image, (classification_targets,
            regression_targets,
            background_mask,
            ignore_mask) = batch
    regression_predictions, classification_predictions = model(
        image, training=False)
    Lreg, Lcls = loss_fn(classification_targets,
                         classification_predictions,
                         regression_targets,
                         regression_predictions,
                         background_mask,
                         ignore_mask)
    total_loss = Lreg + Lcls
    loss_dict = {
        'box_loss': Lreg,
        'cls_loss': Lcls,
        'total_loss': total_loss
    }
    return loss_dict

In [14]:
for ep in range(EPOCHS):
    for batch in tqdm_notebook(input_fn(training=True)()):
        pass
    for batch in tqdm_notebook(input_fn(training=False)()):
        pass

KeyboardInterrupt: 

In [ ]:
def draw_bboxes(image, bbox_list):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    h, w = image.shape.as_list()[:2]
    bboxes = tf.cast(tf.stack([
        bbox_list[:, 1] / h, bbox_list[:, 0] /
        w, bbox_list[:, 3] / h, bbox_list[:, 2] / w
    ], axis=-1), dtype=tf.float32)
    # To do, set colors for each class
    colors = tf.random.uniform(maxval=1, shape=[bbox_list.shape[0], 3])
    return tf.image.draw_bounding_boxes(image[None, ...],
                                        bboxes[None, ...], colors)[0, ...]

def decode_targets(classification_outputs, regression_outputs, input_shape=None, classification_threshold=0.05, nms_threshold=0.5):
    anchors = get_anchors(input_shape=input_shape, tensor=True)
    confidence_scores = tf.reduce_max(
        tf.sigmoid(classification_outputs), axis=-1)
    class_ids = tf.argmax(classification_outputs, axis=-1)
    boxes = tf.concat([(regression_outputs[:, :2] * anchors[:, 2:] + anchors[:, :2]),
                       tf.math.exp(regression_outputs[:, 2:]) * anchors[:, 2:]
                       ], axis=-1)
    non_zero_class_mask = tf.where(class_ids > 0)[:, 0]
    non_zero_class_ids = tf.gather(class_ids, non_zero_class_mask)
    non_zero_class_confidence_scores = tf.gather(
        confidence_scores, non_zero_class_mask)
    non_zero_class_bboxes = tf.gather(boxes, non_zero_class_mask)

    nms_indices = tf.image.non_max_suppression(non_zero_class_bboxes,
                                               non_zero_class_confidence_scores,
                                               iou_threshold=nms_threshold,
                                               max_output_size=200)

    final_class_ids = tf.gather(non_zero_class_ids, nms_indices)
    final_scores = tf.gather(non_zero_class_confidence_scores, nms_indices)
    final_boxes_ = tf.gather(non_zero_class_bboxes, nms_indices)
    final_boxes = tf.cast(change_box_format(final_boxes_, return_format='x1y1x2y2'),
                          dtype=tf.int32)
    return final_boxes, final_class_ids, final_scores